# Recommender Systems

#### First I imported the necessary libraries and the data. I also implemented a couple objects and some utils functions.

In [ ]:
# Import necessary libraries
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Firstly let's create training and testing files
with open('ml-latest-small/ratings.csv', 'r') as input_file:
    # Create two output files for training and testing
    with open('ml-latest-small/ratings_train.csv', 'w', newline='') as training_file, open('ml-latest-small/ratings_test.csv', 'w', newline='') as testing_file:
        # Create CSV writers for the output files
        training_writer = csv.writer(training_file)
        testing_writer = csv.writer(testing_file)
        
        # Loop over the input file and split the rows into the training and testing files
        for i, row in enumerate(csv.reader(input_file)):
            if i % 2 == 0:
                # Even rows go to the training file
                training_writer.writerow(row)
            else:
                # Odd rows go to the testing file
                testing_writer.writerow(row)

In [ ]:
class User:
    def __init__(self, user_id, genres_count):
        super().__init__()
        self.id = user_id
        self.genre_ratings = np.zeros(genres_count, dtype=int)  # Ratings of genres by the user, normalized to (0,1)
        self.ratings: Dict[int, float] = {}  # Star-rating of given movies
        self.movies_rated = set()

class Movie:
    def __init__(self, movie_id, title, genres):
        super().__init__()
        self.id = movie_id
        self.title = title
        self.genres = genres
        self.genres_vector = None

    def set_genres(self, genres_str_int):
        self.genres_vector = np.zeros(len(genres_str_int))
        for genre in self.genres:
            genre_id = genres_str_int[genre]
            self.genres_vector[genre_id] = 1

        self.genres_vector = self.genres_vector.reshape(1, -1)


In [ ]:
# Input part


# import movies from the file. Output format is a tuple (dictionary of Movie objects, list of genres)
def read_movies(path):
    genres_set = set() # initially a set so I don't have to check for duplicates
    movies = {}
    inverted_movies = {}

    with open(path, encoding="utf-8") as f:
        #f.readline() # skip the header
        f.readline()
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            # print("Row: {}".format(row))
            movie_id = int(row[0])
            title = row[1]
            genres = [genre.strip() for genre in row[2].split('|') if genre.strip() != '(no genres listed)']
            movie = Movie(movie_id, title, genres)
            movies[movie_id] = movie
            inverted_movies[title] = movie_id
            genres_set = genres_set.union(set(genres))

    return movies, sorted(list(genres_set)), inverted_movies
    
# import users ratings from the file. Output format is a dictionary of User objects (key is the id)
def read_ratings(path, genre_str_id, movies):
    users = {}
    with open(path, encoding="utf-8") as f:
        f.readline() # skip the header
        f.readline()
        reader = csv.reader(f, delimiter=',')
        for i, row in enumerate(reader):
            user_id = int(row[0])
            movie_id = int(row[1])
            rating = float(row[2])
            # create new user if necessary
            if user_id not in users:
                users[user_id] = User(user_id, len(genre_str_id))
            # add rating if user already present
            users[user_id].movies_rated.add(movie_id)
            users[user_id].ratings[movie_id] = rating
            # +1 to the genre field of the user profile if rating is >= 2.5
            for movie_genre in movies[movie_id].genres:
                if rating >= 2.5:
                    users[user_id].genre_ratings[genre_str_id[movie_genre]] += 1

    # Normalize scores
    for user in users.values():
        user.genre_ratings = user.genre_ratings / 5
    
    return users


In [ ]:
# Evaluation measures
def compute_recall(recommended_movies, rated_movies):
    count = 0
    for movie_id in recommended_movies:
        if movie_id in rated_movies:
            count += 1

    return count / len(rated_movies)


def compute_precision(recommended_movies, rated_movies):
    count = 0
    for movie_id in recommended_movies:
        if movie_id in rated_movies:
            count += 1

    return count / len(recommended_movies)


def compute_fmeasure(precision, recall):
    if precision == 0 and recall == 0:
        return 0

    return 2 * (precision * recall) / (precision + recall)

def evaluate_results(recommended_movies, testing_user):
    
    precision = compute_precision(recommended_movies, testing_user.movies_rated)
    recall = compute_recall(recommended_movies, testing_user.movies_rated)
    fmeasure = compute_fmeasure(precision, recall)

    print("Statistics about recommendation for user {}".format(testing_user.id))
    print("Precision = {:.3f}".format(precision))
    print("Recall = {:.3f}".format(recall))
    print("F-Measure = {:.3f}".format(fmeasure))
    
    return (precision, recall, fmeasure)


## Implementation of content based, collaborative filtering based and hybrid recommender

In [ ]:

# CONTENT BASED RECOMMENDER
def recommend_content_based(movies, users, user_id, N):
    similarities = {}

    # focus only on movies that has not been rated yet by the user
    non_rated_movies = [movie for movie in movies.values() if movie.id not in users[user_id].movies_rated]
    # compute similarity
    for movie in non_rated_movies:
        similarities[movie.id] = cosine_similarity(users[user_id].genre_ratings.reshape(1, -1), movie.genres_vector)[0][0]

    # Sort similarities
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    
    return sorted_similarities[:N]
    
    
# COLLABORATIVE FILTERING BASED RECOMMENDER
def recommend_collaborative_based(users, user_id, N, Nu):
    this_user = users[user_id]

    # compute similarities with other users
    similarities = {}
    for user in users.values():
        if user_id != user.id:
            similarities[user.id] = cosine_similarity(this_user.genre_ratings.reshape(1, -1), user.genre_ratings.reshape(1, -1))[0][0]

    # Get top Nu most similar users
    sorted_similar_users = sorted(similarities.items(), key=lambda i: i[1], reverse=True)[:Nu]

    # Get movies to recommend (their rating is the average of the rating given by the similar users)
    temp_ratings = {}  # {movie_id: list of ratings}
    for user, user_similarity in [(users[u_id], sim) for u_id, sim in sorted_similar_users]:   
        for movie_id, movie_rating in user.ratings.items():  # Iterate through the Users' ratings
            # Focus only on movies that the user hasn't rated
            if movie_id not in this_user.movies_rated:    
                if movie_id not in temp_ratings:
                    temp_ratings[movie_id] = []
                # Add rating (weighted to the similarity)
                temp_ratings[movie_id].append(movie_rating * user_similarity)

    # Calculate average
    avg_ratings = {movie_id: sum(list_rating) / len(list_rating) for movie_id, list_rating in temp_ratings.items()}
    
    # Sort ratings
    sorted_ratings = sorted(avg_ratings.items(), key=lambda r: r[1], reverse=True)
    
    # Normalize ratings
    final_ratings = [(rating[0], (rating[1] / 5)) for rating in sorted_ratings]

    # Select first N (they are sorted)
    res = {}
    for idx, (k, v) in enumerate(final_ratings):
        if idx == N: break
        res[k] = v
    
    return res
    
    
# HYBRID BASED RECOMMENDER     
def recommend_hybrid_based(movies, users, user_id, content_weight, collab_weight, N, Nu):
    content_recommendations = recommend_content_based(movies, users, user_id, N)
    content_recommendations = {t[0]: t[1] for t in content_recommendations}
    collab_recommendations = recommend_collaborative_based(users, user_id, N, Nu)

    # Get all movie ids avoiding duplicates thanks to 'set'
    keys = list(set(list(content_recommendations.keys()) + list(collab_recommendations.keys())))

    final_recommendations = {}
    for key in keys:
        # if movie is recommended by both recommenders, compute weighted score
        if key in content_recommendations and key in collab_recommendations:
            final_recommendations[key] = content_recommendations[key] * content_weight + collab_recommendations[key] * collab_weight
         # else use the score from the only recommender that suggested it
        elif key in content_recommendations and key not in collab_recommendations:
            final_recommendations[key] = content_recommendations[key] * content_weight
        else:
            final_recommendations[key] = collab_recommendations[key] * collab_weight
    
    # Sort in descending order
    sorted_final = sorted(final_recommendations.items(), key=lambda item: item[1], reverse=True)
        
    return sorted_final[:N]
        

#### Let's perform the task with the first three recommenders

In [ ]:
# Parameters
N = 50      # number of results (number of recommendations)
Nu = 20      # number of users to be kept in consideration in the collaborative filtering based recommender

# Import files  
movies, genres_list, inverted_movies = read_movies('ml-latest-small/movies.csv')

# useful dictionaries
genre_str_id = {genre: i for i, genre in enumerate(genres_list)}
genre_id_str = {i: genre for i, genre in enumerate(genres_list)}

for movie in movies.values():
    movie.set_genres(genre_str_id)


users = read_ratings('ml-latest-small/ratings.csv', genre_str_id, movies)

#res = recommend_content_based(movies, users, 1, N)
#for r in res:
#    print(movies[r[0]].title)
    
#res = recommend_collaborative_based(users, 100, N, Nu)
#for r in res.keys():
#    print(movies[r].title)

#res = recommend_hybrid_based(movies, users, 1, 0.3, 0.7, N, Nu)
#for r in res:
   #print(movies[r[0]].title)



In [ ]:
users_train = read_ratings('ml-latest-small/ratings_train.csv', genre_str_id, movies)
users_test = read_ratings('ml-latest-small/ratings_test.csv', genre_str_id, movies)
id_test = 11
test_user = users_test[id_test]

In [ ]:
print("CONTENT BASED RECOMMENDER\n")
recommended_movies = recommend_content_based(movies, users_train, id_test, N)
recommended_movies = [item[0] for item in recommended_movies]
res = evaluate_results(recommended_movies, test_user)

In [ ]:
print("COLLABORATIVE FILTERING BASED RECOMMENDER\n")
recommended_movies = recommend_collaborative_based(users_train, id_test, N, Nu)
res = evaluate_results(list(recommended_movies.keys()), test_user)

In [ ]:
print("HYBRID RECOMMENDER\n")
weights = [(0.1, 0.9), (0.3, 0.7), (0.7, 0.3), (0.9, 0.1)]
for (content_weight, collab_weight) in weights:
    print("-"*70)
    print("Content based weight: {}".format(content_weight))
    print("Collaborative filtering based weight: {}\n".format(collab_weight))
    recommended_movies = recommend_hybrid_based(movies, users_train, id_test, content_weight, collab_weight, N, Nu)
    recommended_movies = [item[0] for item in recommended_movies]
    res = evaluate_results(recommended_movies, test_user)
    print()

## Comments

#### The parameters I used are: number of movies to recommend = 50 and number of similar users to consider = 20. I ran the notebook for 5 randomly selected users. These parameters are also valid for the second part. The results for the content based recommender and the collaborative filtering based recommender can be seen in the 'results_base.csv' file. The results for the hybrid recommender can be seen in the 'results_hybrid.csv' file. Other than the results in the files I did more experiments and I've seen that the content based recommender worked better than the collaborative filtering one, even if they both had bad results overall. Or at least I was expecting better scores. Anyway one thing that I noticed is that content based and collaborative filtering recommended different movies and for this reason the hybrid recommender basically became a content based or a collaborative filtering one, depending on the weights. This is because of my implementation: if a movie is recommended by just one of the two recommenders its similarity gets scaled down by the corresponding weight factor, so basically if for example the weights are 0.1 and 0.9, movies recommended just by the model with a weight of 0.1 will never be selected.

## K-Nearest Neighbors based recommender

#### The reason why I chose this model instead of Tensorflow recommenders can be found in the 'issues' section.
#### Anyway, I first rearranged the data in order to have a table where each row is a user, each column a movie and the values are the ratings (zero if not rated yet)

In [ ]:
ratings_dataset = pd.read_csv('ml-latest-small/ratings_train.csv', sep=',', header=0)
del ratings_dataset['timestamp']
ratings_dataset.head()

In [ ]:
column_names = ['movie id', 'title', 'genres']
movies_dataset = pd.read_csv('ml-latest-small/movies.csv', sep=',', header=0)
movies_dataset.head()

In [ ]:
genres_dataset = items_dataset[['movieId', 'genres']]
genres_list = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi','Thriller', 'War', 'Western', 
                    'IMAX', '(no genres listed)']
for genre in genres_list:
    genres_dataset[genre] = [0] * 9742 # (number of movies)

for row in genres_dataset.iterrows():
    index = row[0]
    content = row[1]
    film_genres = content['genres'].split('|')
    for g in film_genres:
        genres_dataset.iat[index, genres_dataset.columns.get_loc(g)] = 1

del genres_dataset['genres']
genres_dataset.head()

In [ ]:
processed_movies = pd.merge(
    movies_dataset,
    genres_dataset,
    how="inner",
    on='movieId',
)
del processed_movies['genres']
del processed_movies['(no genres listed)']
processed_movies.head()

In [ ]:
merged_dataset = pd.merge(ratings_dataset, processed_movies, how='inner', on='movieId')
merged_dataset.head()

In [ ]:
refined_dataset = merged_dataset.groupby(by=['userId','title'], as_index=False).agg({"rating":"mean"})
user_to_movie_df = refined_dataset.pivot(
    index='userId',
    columns='title',
    values='rating').fillna(0)

user_to_movie_df.sample(n=10)

#### Now let's perform the actual task

In [ ]:
# transform matrix to scipy sparse matrix
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)

# Define and fit the model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_to_movie_sparse_df)

In [ ]:
def get_similar_users(user, n):
  knn_input = np.asarray([user_to_movie_df.values[user-1]]) 
  distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
  print("Top", n, "most simlar users to the user", user," are: \n")
  for i in range(1,len(distances[0])):
    print("{}. User {} --> {:.3f}".format(i, indices[0][i]+1, distances[0][i]))
  
  return indices.flatten()[1:] + 1, distances.flatten()[1:]


def filtered_movie_recommendations(n, mean_rating_list, refined_dataset, user_id):
  
  # Find the index of the last occurrence of 0 in mean_rating_list
  zero_index = np.where(mean_rating_list == 0)[0][-1]

  # Sort the mean_rating_list in descending order
  sorted_indices = np.argsort(mean_rating_list)[::-1]

  # Select only the indices before zero_index
  filtered_indices = sorted_indices[:list(sorted_indices).index(zero_index)]

  # Limit the number of indices to n
  n_filtered_indices = min(len(filtered_indices), n)

  # Get a list of movies watched by user_id
  movies_watched = list(refined_dataset[refined_dataset['userId'] == user_id]['title'])

  # Create a filtered list of movies based on the filtered_indices
  filtered_movie_list = list(movies_list[filtered_indices[:n_filtered_indices]])

  count = 0
  final_movie_list = []
  for m in filtered_movie_list:
    if m not in movies_watched:
      count += 1
      final_movie_list.append(m)
    if count == n:
      break
  
  if count == 0:
    print("No movies which are not seen by the input user and seen by similar users. Please increase the number of similar users.")
  elif count < n:
    print("There aren't {} movies which are not seen by the input user and seen by similar users.".format(n))
    print("Anyway, I found {} to recommend. If you want {}, please increase the number of similar users to consider.\nMovies:".format(count, n))
    for i, movie in zip(range(1,len(final_movie_list)+1), final_movie_list):
      print(i,". ",movie)
  else:
    for i, movie in zip(range(1,len(final_movie_list)+1), final_movie_list):
      print(i,". ",movie) 
  
  return final_movie_list

#### At first we find the most similar users (and their similarity)

In [ ]:
# Parameters
#id_test = 1
num_users = 20
num_movies = 50
testing_user = users_test[id_test]


similar_users_list, similarities_list = get_similar_users(user_id, num_users)

#### Then we get the recommended movies. The "role" of a similar user in the recommendation task is weighted to his similarity with the main user.

In [ ]:
weight_list = similarities_list / np.sum(similarities_list)

# Movie ratings from similar users
movie_ratings = user_to_movie_df.values[similar_users_list]
movies_list = user_to_movie_df.columns

# Broadcasting weight matrix to rating matrix so it is compatible for matrix operations
weight_list = weight_list[:,np.newaxis] + np.zeros(len(movies_list))
weighted_rating_matrix = weight_list * movie_ratings
mean_rating_list = weighted_rating_matrix.sum(axis = 0)

recommended_movies = filtered_movie_recommendations(num_movies, mean_rating_list, refined_dataset, user_id)

In [ ]:
recommended_movies = [inverted_movies[title] for title in recommended_movies]
res = evaluate_results(recommended_movies, testing_user)

## Comments

#### The results can be seen in the 'results_knn.csv' file. Using this model I got better results than the previous ones, but I was still expecting better scores. 

## Issues and posible extensions

#### For the first part no major issues. In the second part I haven't been able to use Tensorflow recommenders because every time I tried to import it the kernel crashed. I had a similar issue in a previous homework but this time I didn't managed to solve it. For this reason I didn't follow the suggestion in the task description and I implemented a recommender based on sklearn k-nearest neighbors. A possible extension could be modifying the second part in order for the knn model to be able to compute the distance between users focusing not only on the rating they gave to movies but also the genre infromation, that isn't used in my implementation.